# Lotte Data

## Preprocessing

In [0]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 481kB 9.1MB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhins2rk/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm
import json
from google.colab import drive 
import urllib.request
from pandas.io.json import json_normalize
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
prod = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Pruduct.csv')
mast = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Master.csv')
sess = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Session.csv')
cust = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Custom.csv')
# sear = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Search1.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 자료형 변환
- **object** to **integer**

In [19]:
prod['PD_BUY_AM'] = pd.to_numeric(prod['PD_BUY_AM'].str.replace(',',''))
prod['PD_BUY_CT'][29649] = 1440
prod['PD_BUY_CT'][34105] = 1387
prod['PD_BUY_CT'] = pd.to_numeric(prod['PD_BUY_CT'])

sess['TOT_SESS_HR_V'] = pd.to_numeric(sess['TOT_SESS_HR_V'].str.replace(',',''))
sess['MONTH'] = sess['SESS_DT'].map(lambda x : str(x)[:4] + '-' + str(x)[4:6])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### 브랜드명 수정
- 브랜드명에서 **불필요한 단어** 제거(예. 향수, 괄호, 공백 등)
- **잘못입력된** 브랜드명 수정

In [0]:
prod['PD_BRA_NM'] = prod['PD_BRA_NM'].map(lambda x: x.split('(')[0].replace('[','').replace(']','').replace(' ',''))
prod.loc[prod['PD_BRA_NM']=='MADE IN ITALY','PD_BRA_NM'] = 'V73'
prod.loc[prod['PD_BRA_NM'].isin(['1909535','19330640825']),'PD_BRA_NM'] = 'KL'
prod.loc[prod['PD_BRA_NM']=='19386640906','PD_BRA_NM'] = '지이크'
prod.loc[prod['PD_BRA_NM'].isin(['1907517','1924113','1906469','1906471','1902295','1908178','1881326','1924116','1906492','1902287']),'PD_BRA_NM'] = 'CC collect'
prod.loc[prod['PD_BRA_NM'].isin(['19379580904','19364580830','1872892','1898551','18703070925','19359410830']),'PD_BRA_NM'] = '더아이잗'
prod.loc[prod['PD_BRA_NM'].isin(['1904418','1896342']),'PD_BRA_NM'] = '보니스팍스'
prod.loc[prod['PD_BRA_NM'].isin(['1912506','1912518']),'PD_BRA_NM'] = '리스트'
prod.loc[prod['PD_BRA_NM']=='19326280823','PD_BRA_NM'] = 'ENC'
prod.loc[prod['PD_BRA_NM'].isin(['19266920905','19266920905','19266920905']),'PD_BRA_NM'] = '요하넥스'
prod.loc[prod['PD_BRA_NM']=='1913802','PD_BRA_NM'] = '꼼빠니아'
prod.loc[prod['PD_BRA_NM']=='1917759','PD_BRA_NM'] = '인디안'

In [0]:
prod_mast = pd.merge(prod, mast, how = 'inner', on=['PD_C'])


# df[df['PD_BUY_CT']>=150000].sort_values(by=['PD_BUY_AM'],ascending=False)
cate_list= ['패션잡화','여성의류','화장품/뷰티케어','남성의류']
prod_mast = prod_mast[prod_mast['CLAC1_NM'].isin(cate_list)]

In [0]:
# l = ['18703070925','1872892','1881326','1896342','1898551','1902287','1902295','1904418','1906469','1906471','1906492','1907517','1908178',
#      '1912506','1912518','1913802', '1917759','1924113','1924116','19266920905','19326280823','19359410830','19364580830','19379580904']
# prod_mast[prod_mast['PD_BRA_NM'].isin(l)]

#### 명품브랜드만 골라내기

In [9]:
mp1 = pd.read_csv("gdrive/My Drive/공모전/Lpoint/명품 골라내기/lotte_luxury_list.csv")
brand_lotte = list(mp1['BRA_NM'])

brand_lotte[:100:10],len(brand_lotte)

(['티르리르',
  '클라란스',
  '겐조',
  'CHLOE',
  '커먼프로젝트',
  '엘리자베스아덴',
  '휴고보스',
  '마르니',
  'RMK',
  '몬데인'],
 275)

In [13]:
mp2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/명품 골라내기/my_luxury_list.csv')
brands = brand_lotte
brands.extend(mp2['title'])
print(mp2['title'][:100:10], len(mp2['title']))
brands = list(set(brands))


0      브룩스브라더스
10         멀버리
20         랭앤루
30         로즈몽
40    비비안웨스트우드
50        오일릴리
60          달팡
70        토리버치
80       만다리나덕
90     케이트스페이드
Name: title, dtype: object 269


In [16]:
brands[1:300:30],len(brands)

(['톰포드',
  '발렌시아가',
  '순토',
  '프레쉬',
  '올리브데올리브',
  '오메가',
  '발렉스트라',
  '샹테카이',
  '골든구스',
  '더센트오프디파쳐'],
 463)

In [28]:
prod_mast = prod_mast[prod_mast['PD_BRA_NM'].isin(brands)]
prod_mast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727959 entries, 0 to 5024890
Data columns (total 12 columns):
CLNT_ID      727959 non-null int64
SESS_ID      727959 non-null int64
HITS_SEQ     727959 non-null int64
PD_C         727959 non-null int64
PD_ADD_NM    727959 non-null object
PD_BRA_NM    727959 non-null object
PD_BUY_AM    727959 non-null int64
PD_BUY_CT    727959 non-null int64
PD_NM        727959 non-null object
CLAC1_NM     727959 non-null object
CLAC2_NM     727959 non-null object
CLAC3_NM     727959 non-null object
dtypes: int64(6), object(6)
memory usage: 72.2+ MB


In [31]:
prod_mast['PD_BRA_NM'].drop_duplicates()[:100:10], len(prod_mast['PD_BRA_NM'].drop_duplicates())

(0           바비브라운
 4348        메트로시티
 6078            랩
 6463     크로커다일레이디
 9154          포커스
 9671          비오템
 11540    까르뜨블랑슈남성
 12939       레쁘띠뜨-
 14416       에고이스트
 22185        토니모리
 Name: PD_BRA_NM, dtype: object, 4201)

### Merge data

In [0]:
pmc = pd.merge(prod_mast, cust, how = 'left', on=['CLNT_ID'])
pms = pd.merge(pmc, sess, how = 'inner', on=['CLNT_ID','SESS_ID'])

In [0]:
pms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651270 entries, 0 to 651269
Data columns (total 22 columns):
CLNT_ID            651270 non-null int64
SESS_ID            651270 non-null int64
HITS_SEQ           651270 non-null int64
PD_C               651270 non-null int64
PD_ADD_NM          651270 non-null object
PD_BRA_NM          651270 non-null object
PD_BUY_AM          651270 non-null int64
PD_BUY_CT          651270 non-null int64
PD_NM              651270 non-null object
CLAC1_NM           651270 non-null object
CLAC2_NM           651270 non-null object
CLAC3_NM           651270 non-null object
CLNT_GENDER        524795 non-null object
CLNT_AGE           524795 non-null float64
SESS_SEQ           651270 non-null int64
SESS_DT            651270 non-null int64
TOT_PAG_VIEW_CT    651227 non-null float64
TOT_SESS_HR_V      648982 non-null float64
DVC_CTG_NM         651270 non-null object
ZON_NM             651270 non-null object
CITY_NM            651270 non-null object
MONTH       

## By Product

### Price

In [0]:
df_pm = pms.groupby(['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).reset_index()
df_pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16571 entries, 0 to 16570
Data columns (total 7 columns):
MONTH        16571 non-null object
CLAC1_NM     16571 non-null object
CLAC2_NM     16571 non-null object
CLAC3_NM     16571 non-null object
PD_BRA_NM    16571 non-null object
PD_BUY_AM    16571 non-null float64
PD_BUY_CT    16571 non-null int64
dtypes: float64(1), int64(1), object(5)
memory usage: 906.3+ KB


In [0]:
df_pm['PD_BUY_AM'] = round(df_pm['PD_BUY_AM']).map(lambda x: int(x))
df_pm.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,239200,1
1,2018-04,남성의류,남성의류상의,남성가디건,게스,72507,43
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,77400,1
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,231750,16
4,2018-04,남성의류,남성의류상의,남성가디건,본,50000,1


### Sales

In [0]:
df_pm['PD_SALES'] = df_pm['PD_BUY_AM']*df_pm['PD_BUY_CT']
df_pm.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_SALES
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,239200,1,239200
1,2018-04,남성의류,남성의류상의,남성가디건,게스,72507,43,3117801
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,77400,1,77400
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,231750,16,3708000
4,2018-04,남성의류,남성의류상의,남성가디건,본,50000,1,50000


## By Customer


### Gender & Age

In [0]:
pms['GENDER_AGE'] = pms[['CLNT_GENDER','CLNT_AGE']].apply(lambda x: '{}_{}'.format(x[0],x[1]), axis=1)

In [0]:
g_pmc = pms.groupby(['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','GENDER_AGE']).agg({'CLNT_ID':'count'}).reset_index()
g_pmc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63050 entries, 0 to 63049
Data columns (total 7 columns):
MONTH         63050 non-null object
CLAC1_NM      63050 non-null object
CLAC2_NM      63050 non-null object
CLAC3_NM      63050 non-null object
PD_BRA_NM     63050 non-null object
GENDER_AGE    63050 non-null object
CLNT_ID       63050 non-null int64
dtypes: int64(1), object(6)
memory usage: 3.4+ MB


In [0]:
g_pmc.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,GENDER_AGE,CLNT_ID
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,M_50.0,1
1,2018-04,남성의류,남성의류상의,남성가디건,게스,F_20.0,1
2,2018-04,남성의류,남성의류상의,남성가디건,게스,F_30.0,5
3,2018-04,남성의류,남성의류상의,남성가디건,게스,F_40.0,23
4,2018-04,남성의류,남성의류상의,남성가디건,게스,F_50.0,3


In [0]:
df_pmc = g_pmc.pivot_table(values='CLNT_ID',index=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='GENDER_AGE').fillna(0).reset_index()
df_pmc.head()

GENDER_AGE,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,F_10.0,F_20.0,F_30.0,F_40.0,F_50.0,...,F_80.0,M_10.0,M_20.0,M_30.0,M_40.0,M_50.0,M_60.0,M_70.0,M_80.0,nan_nan
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,0.0,1.0,5.0,23.0,3.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,9.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,0.0,3.0,4.0,1.0,3.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Sign in

In [0]:
df_pmc.columns = ['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'F_10', 'F_20', 'F_30', 'F_40',
       'F_50', 'F_60', 'F_70', 'F_80', 'M_10', 'M_20', 'M_30',
       'M_40', 'M_50', 'M_60', 'M_70', 'M_80', 'NOT_SIGNED']
df_pmc.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,F_10,F_20,F_30,F_40,F_50,...,F_80,M_10,M_20,M_30,M_40,M_50,M_60,M_70,M_80,NOT_SIGNED
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,0.0,1.0,5.0,23.0,3.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,9.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,0.0,3.0,4.0,1.0,3.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## By Session

### Location

In [0]:
g_pms_l = pms.groupby(['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','ZON_NM']).agg({'CLNT_ID':'count'}).reset_index()
g_pms_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85906 entries, 0 to 85905
Data columns (total 7 columns):
MONTH        85906 non-null object
CLAC1_NM     85906 non-null object
CLAC2_NM     85906 non-null object
CLAC3_NM     85906 non-null object
PD_BRA_NM    85906 non-null object
ZON_NM       85906 non-null object
CLNT_ID      85906 non-null int64
dtypes: int64(1), object(6)
memory usage: 4.6+ MB


In [0]:
df_pms_l = g_pms_l.pivot_table(values='CLNT_ID',index=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='ZON_NM').fillna(0).reset_index()
df_pms_l.head()

ZON_NM,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,Busan,Chungcheongbuk-do,Chungcheongnam-do,Daegu,Daejeon,...,Gwangju,Gyeonggi-do,Gyeongsangbuk-do,Gyeongsangnam-do,Incheon,Jeju-do,Jeollabuk-do,Jeollanam-do,Seoul,Ulsan
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,0.0,0.0,8.0,5.0,0.0,...,0.0,7.0,1.0,3.0,1.0,0.0,1.0,0.0,16.0,1.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,3.0,0.0,0.0,3.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df_pms_l.columns = ['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'Busan', 'Chungbuk',
       'Chungnam', 'Daegu', 'Daejeon', 'Gangwon', 'Gwangju',
       'Gyeonggi', 'Gyeongbu', 'Gyeongnam', 'Incheon',
       'Jeju', 'Jeonbuk', 'Jeonnam', 'Seoul', 'Ulsan']
df_pms_l.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,Busan,Chungbuk,Chungnam,Daegu,Daejeon,...,Gwangju,Gyeonggi,Gyeongbu,Gyeongnam,Incheon,Jeju,Jeonbuk,Jeonnam,Seoul,Ulsan
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,0.0,0.0,8.0,5.0,0.0,...,0.0,7.0,1.0,3.0,1.0,0.0,1.0,0.0,16.0,1.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,3.0,0.0,0.0,3.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Device

In [0]:
g_pms_d = pms.groupby(['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM','DVC_CTG_NM']).agg({'CLNT_ID':'count'}).reset_index()
g_pms_d.loc[g_pms_d['DVC_CTG_NM'] == 'tablet','DVC_CTG_NM'] = 'mobile'
g_pms_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27256 entries, 0 to 27255
Data columns (total 7 columns):
MONTH         27256 non-null object
CLAC1_NM      27256 non-null object
CLAC2_NM      27256 non-null object
CLAC3_NM      27256 non-null object
PD_BRA_NM     27256 non-null object
DVC_CTG_NM    27256 non-null object
CLNT_ID       27256 non-null int64
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [0]:
df_pms_d = g_pms_d.pivot_table(values='CLNT_ID',index=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'],columns='DVC_CTG_NM').fillna(0).reset_index()
df_pms_d.head()

DVC_CTG_NM,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,desktop,mobile
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,1.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,9.0,34.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,1.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,2.0,14.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,0.0,1.0


In [0]:
df_pms_d.columns = ['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM', 'PD_BRA_NM', 'Desktop', 'Mobile']
df_pms_d.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,Desktop,Mobile
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,1.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,9.0,34.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,1.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,2.0,14.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,0.0,1.0


### Merge

In [0]:
df_pms = pd.merge(df_pms_d,df_pms_l, how='inner', on=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
df_pms.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,Desktop,Mobile,Busan,Chungbuk,Chungnam,...,Gwangju,Gyeonggi,Gyeongbu,Gyeongnam,Incheon,Jeju,Jeonbuk,Jeonnam,Seoul,Ulsan
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04,남성의류,남성의류상의,남성가디건,게스,9.0,34.0,0.0,0.0,8.0,...,0.0,7.0,1.0,3.0,1.0,0.0,1.0,0.0,16.0,1.0
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,2.0,14.0,3.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0
4,2018-04,남성의류,남성의류상의,남성가디건,본,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Merge ALL

In [0]:
data =  pd.merge(df_pm,df_pmc, how='inner', on=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
data =  pd.merge(data,df_pms, how='inner', on=['MONTH','CLAC1_NM','CLAC2_NM','CLAC3_NM','PD_BRA_NM'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16571 entries, 0 to 16570
Data columns (total 43 columns):
MONTH         16571 non-null object
CLAC1_NM      16571 non-null object
CLAC2_NM      16571 non-null object
CLAC3_NM      16571 non-null object
PD_BRA_NM     16571 non-null object
PD_BUY_AM     16571 non-null int64
PD_BUY_CT     16571 non-null int64
PD_SALES      16571 non-null int64
F_10          16571 non-null float64
F_20          16571 non-null float64
F_30          16571 non-null float64
F_40          16571 non-null float64
F_50          16571 non-null float64
F_60          16571 non-null float64
F_70          16571 non-null float64
F_80          16571 non-null float64
M_10          16571 non-null float64
M_20          16571 non-null float64
M_30          16571 non-null float64
M_40          16571 non-null float64
M_50          16571 non-null float64
M_60          16571 non-null float64
M_70          16571 non-null float64
M_80          16571 non-null float64
NOT_SIGNED    

In [0]:
data.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/mlotte.csv',index=False)